# Spaces and forms on subdomains
     

In NGSolve, finite element spaces can be defined on subdomains. This is useful for multiphysics problems like fluid-structure interaction.

In addition, bilinear or linear forms can be defined as integrals over regions. *Regions* are parts of the domain. They may be subdomains, or  parts of the domain boundary, or parts of subdomain interfaces.


In [1]:
import netgen.gui
%gui tk
from netgen.geom2d import *
from ngsolve import *

### Naming subdomains and boundaries

We define a geometry with multiple regions and assign names to these regions below:

In [2]:
geo = SplineGeometry()
geo.AddRectangle((0,0), (2,2),
                 bcs=["b","r","t","l"],
                 leftdomain=1)
geo.AddRectangle((1,0.9), (1.3,1.4),
                 bcs=["b2","r2","t2","l2"],
                 leftdomain=2, rightdomain=1)
geo.SetMaterial (1, "outer")
geo.SetMaterial (2, "inner")
mesh = Mesh(geo.GenerateMesh(maxh=0.1))

bcs =  ['b', 'r', 't', 'l']
bcs =  ['b2', 'r2', 't2', 'l2']



* These statements define two rectangular subdomain regions named "inner", "outer".

* The bottom, right, top and left parts of the outer rectangle's boundaries define boundary regions, respectively labeled  "b", "r", "t", "l".

* Similarly, the bottom, right, top and left parts of the inner rectangle's boundaries are regions named "b2", "r2", "t2", "l2".

* You can see the subdomain outlines in the Netgen window when you select the `Geometry` tab. When you select the `Mesh` tab and double click on a point, the two subdomains are rendered in different colors.


### A finite element space on a subdomain

In [4]:
fes1 = H1(mesh, definedon="inner")

u1 = GridFunction(fes1, "u1")
u1.Set(x*y)
Draw(u1)

Note how $u_1$ is displayed only in the `inner` region. 

### Integrating on regions 

You have already seen how boundary regions are used in setting Dirichlet boundary conditions.  

In [27]:
fes = H1(mesh, order=3, dirichlet="b|l|r")

u = fes.TrialFunction()
v = fes.TestFunction()
gfu = GridFunction(fes)

Boundary regions or subdomains can also serve as domains of integration.

In [25]:
f = LinearForm(fes)
f += SymbolicLFI(u1*v, definedon=mesh.Materials("inner"))
f += SymbolicLFI(0.1*v, definedon=mesh.Boundaries("t"))
f.Assemble()

Here the functional $f$ is defined as a sum of two integrals, one over the inner subdomain, and another over the top boundary:
$$
f(v) = \int_{\Omega_{inner}} u_1 v \; dx + \int_{\Gamma_{top}} \frac{v}{10}\; ds
$$

*Note:* `SymbolicLFI` has no information about the mesh. Hence its not  enough to  give it the just the name of the region ("inner", or "t").  We must give it the region objects (named by the "inner" and "t" labels) using the `definedon` flag.


In [29]:
a = BilinearForm(fes)
a += SymbolicBFI(grad(u)*grad(v))
a.Assemble()
# Solve the problem:
gfu.vec.data = a.mat.Inverse(fes.FreeDofs()) * f.vec
Draw (gfu)

### More about region objects

In [49]:
mesh.GetMaterials()   # list all subdomains

['outer', 'inner']

In [50]:
mesh.GetBoundaries() # list boundary/interface regions

['b', 'r', 't', 'l', 'b2', 'r2', 't2', 'l2']

In [51]:
mesh.Materials("inner") # look at object's type 

In [52]:
mesh.Boundaries("t")   # same type

### Operations with regions

Print region information:

In [55]:
print(mesh.Materials("inner").Mask())
print(mesh.Materials("[a-z]*").Mask())  # can use regexp
print(mesh.Boundaries('t|l').Mask())

0: 01
0: 11
0: 00110000


Add regions:

In [57]:
io = mesh.Materials("inner") + mesh.Materials("outer")
print(io.Mask())

0: 11


Take complement of a region:

In [59]:
c = ~mesh.Materials("inner")
print(c.Mask())

0: 10


Subtract regions:

In [60]:
diff = mesh.Materials("inner|outer") - mesh.Materials("outer")
print(diff.Mask())

0: 01


Set piecewise coefficients based on subdomains:

In [67]:
domain_values = {'inner': 3.7,  'outer': 1}
values_list = [domain_values[mat]
               for mat in mesh.GetMaterials()]
cf = CoefficientFunction(values_list)
Draw(cf, mesh, 'piecewise')

Coefficients on boundary regions are given similarly. Let's make a function that is one at the bottom right vertex of current domain (0,2) x (0,2).

In [72]:
bdry_values = {'b': x, 'r': 2-y}
values_list = [bdry_values[bc]
               if bc in bdry_values.keys() else 0
               for bc in mesh.GetBoundaries()]
cf = CoefficientFunction(values_list)
Draw(cf, mesh, 'piecewise')

What happened here?!

Note that `cf` has no information on boundary regions, so it cannot yet associate the list of values to boundary regions. By default, the list of values are assumed to be subdomain values.

To associate these values to boundary regions, we use a `GridFunction` and `Set`, which also lets us view an extension of these boundary values into the domain.

In [80]:
g = GridFunction(H1(mesh), name='bdry')
g.Set(cf, definedon=~mesh.Boundaries(''))
Draw(g, max=2, min=0)

If you think that specifying the whole boundary using

    ~mesh.Boundaries('')

is convoluted, then you may say 

    g.Set(cf, definedon=~mesh.Boundaries(''))
    
What happens if you used 

    g.Set(cf, BND)
    
instead?  See below!
 

### Dirichlet regions

### Unused dofs

In [83]:
fes1 = H1(mesh, definedon="inner")
fes = H1(mesh)

fes1.ndof, fes.ndof


(486, 486)

In [87]:
print(fes1.FreeDofs())

0: 00001111000000000000000000000000000000000000000000
50: 00000000000000000000000000000000001111111111111100
100: 00000000000000000000000000000000000000000000000000
150: 00000000000000000000000000000000000000000000000000
200: 00000000000000000000000000000000000000000000000000
250: 00000000000000000000000000000000000000000000000000
300: 00000000000000000000000000000000000000000000000000
350: 00000000000000000000000000000000000000000000000000
400: 00000000000000000000000000000000000000000000000000
450: 000000000000000000000000011111111111


In [90]:
for i in range(fes1.ndof):
    print(fes1.CouplingType(i))

COUPLING_TYPE.UNUSED_DOF
COUPLING_TYPE.UNUSED_DOF
COUPLING_TYPE.UNUSED_DOF
COUPLING_TYPE.UNUSED_DOF
COUPLING_TYPE.WIREBASKET_DOF
COUPLING_TYPE.WIREBASKET_DOF
COUPLING_TYPE.WIREBASKET_DOF
COUPLING_TYPE.WIREBASKET_DOF
COUPLING_TYPE.UNUSED_DOF
COUPLING_TYPE.UNUSED_DOF
COUPLING_TYPE.UNUSED_DOF
COUPLING_TYPE.UNUSED_DOF
COUPLING_TYPE.UNUSED_DOF
COUPLING_TYPE.UNUSED_DOF
COUPLING_TYPE.UNUSED_DOF
COUPLING_TYPE.UNUSED_DOF
COUPLING_TYPE.UNUSED_DOF
COUPLING_TYPE.UNUSED_DOF
COUPLING_TYPE.UNUSED_DOF
COUPLING_TYPE.UNUSED_DOF
COUPLING_TYPE.UNUSED_DOF
COUPLING_TYPE.UNUSED_DOF
COUPLING_TYPE.UNUSED_DOF
COUPLING_TYPE.UNUSED_DOF
COUPLING_TYPE.UNUSED_DOF
COUPLING_TYPE.UNUSED_DOF
COUPLING_TYPE.UNUSED_DOF
COUPLING_TYPE.UNUSED_DOF
COUPLING_TYPE.UNUSED_DOF
COUPLING_TYPE.UNUSED_DOF
COUPLING_TYPE.UNUSED_DOF
COUPLING_TYPE.UNUSED_DOF
COUPLING_TYPE.UNUSED_DOF
COUPLING_TYPE.UNUSED_DOF
COUPLING_TYPE.UNUSED_DOF
COUPLING_TYPE.UNUSED_DOF
COUPLING_TYPE.UNUSED_DOF
COUPLING_TYPE.UNUSED_DOF
COUPLING_TYPE.UNUSED_DOF
COUPLING_